In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [4]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [5]:
spark

In [6]:
input = 'apple'
letters = sc.parallelize(input)

print(type(letters))
print(letters.collect())

<class 'pyspark.rdd.RDD'>
['a', 'p', 'p', 'l', 'e']


In [7]:
input = [2,3,4,5,6]
numbers = sc.parallelize(input)
print(numbers.collect())

[2, 3, 4, 5, 6]


In [ ]:
# Transformations

In [11]:
input = ['apple','apple','banana','carrot','carrot','grapes','melon','mango','orange']
words = sc.parallelize(input)
a = words.map(lambda i:(i,1))
b = a.reduceByKey(lambda a,b:a+b)
print(a.collect())
print(b.collect())

[('apple', 1), ('apple', 1), ('banana', 1), ('carrot', 1), ('carrot', 1), ('grapes', 1), ('melon', 1), ('mango', 1), ('orange', 1)]
[('banana', 1), ('carrot', 2), ('melon', 1), ('mango', 1), ('orange', 1), ('apple', 2), ('grapes', 1)]


In [13]:
letters = words.flatMap(lambda a: list(a))
print(letters.collect())

['a', 'p', 'p', 'l', 'e', 'a', 'p', 'p', 'l', 'e', 'b', 'a', 'n', 'a', 'n', 'a', 'c', 'a', 'r', 'r', 'o', 't', 'c', 'a', 'r', 'r', 'o', 't', 'g', 'r', 'a', 'p', 'e', 's', 'm', 'e', 'l', 'o', 'n', 'm', 'a', 'n', 'g', 'o', 'o', 'r', 'a', 'n', 'g', 'e']


In [14]:
c = letters.map(lambda a:(a,1)).reduceByKey(lambda a,b:a+b)
print(c.collect())

[('p', 5), ('l', 3), ('b', 1), ('c', 2), ('r', 6), ('g', 3), ('s', 1), ('a', 10), ('e', 5), ('n', 5), ('o', 5), ('t', 2), ('m', 2)]


In [15]:
d = c.filter(lambda a:a[1]>=5)
print(d.collect())

[('p', 5), ('r', 6), ('a', 10), ('e', 5), ('n', 5), ('o', 5)]


In [74]:
def func(a):
    yield sum(a)

num = sc.parallelize([1,2,3,1,2,3,1,2],3)
e = num.mapPartitions(lambda a:func(a))
print(e.collect())
print(num.glom().collect())

[3, 4, 8]
[[1, 2], [3, 1], [2, 3, 1, 2]]


In [20]:
def index(i,a):
    yield i,sum(a)
f = num.mapPartitionsWithIndex(lambda i,a:index(i,a))
print(f.collect())

[(0, 3), (1, 4), (2, 8)]


In [21]:
a = sc.parallelize('mango')
b = sc.parallelize('apple')
c = a.union(b)
print(c.collect())

['m', 'a', 'n', 'g', 'o', 'a', 'p', 'p', 'l', 'e']


In [22]:
c = a.intersection(b)
print(c.collect())

['a']


In [23]:
print(b.distinct().collect())

['p', 'l', 'a', 'e']


In [25]:
c = letters.map(lambda a:(a,1)).groupByKey()
print(c.collect())

[('p', <pyspark.resultiterable.ResultIterable object at 0x7f37d9771850>), ('l', <pyspark.resultiterable.ResultIterable object at 0x7f37d9771650>), ('b', <pyspark.resultiterable.ResultIterable object at 0x7f37d9771990>), ('c', <pyspark.resultiterable.ResultIterable object at 0x7f37d97718d0>), ('r', <pyspark.resultiterable.ResultIterable object at 0x7f37d9771950>), ('g', <pyspark.resultiterable.ResultIterable object at 0x7f37d9771a50>), ('s', <pyspark.resultiterable.ResultIterable object at 0x7f37d9771a90>), ('a', <pyspark.resultiterable.ResultIterable object at 0x7f37d9771910>), ('e', <pyspark.resultiterable.ResultIterable object at 0x7f37d9771a10>), ('n', <pyspark.resultiterable.ResultIterable object at 0x7f37d9771b90>), ('o', <pyspark.resultiterable.ResultIterable object at 0x7f37d9771ad0>), ('t', <pyspark.resultiterable.ResultIterable object at 0x7f37d9771c90>), ('m', <pyspark.resultiterable.ResultIterable object at 0x7f37d9771cd0>)]


In [29]:
c = letters.map(lambda a:(a,1)).groupByKey().mapValues(list)
print(c.collect())

[('p', [1, 1, 1, 1, 1]), ('l', [1, 1, 1]), ('b', [1]), ('c', [1, 1]), ('r', [1, 1, 1, 1, 1, 1]), ('g', [1, 1, 1]), ('s', [1]), ('a', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), ('e', [1, 1, 1, 1, 1]), ('n', [1, 1, 1, 1, 1]), ('o', [1, 1, 1, 1, 1]), ('t', [1, 1]), ('m', [1, 1])]


In [30]:
c = letters.map(lambda a:(a,1)).groupByKey().mapValues(len)
print(c.collect())

[('p', 5), ('l', 3), ('b', 1), ('c', 2), ('r', 6), ('g', 3), ('s', 1), ('a', 10), ('e', 5), ('n', 5), ('o', 5), ('t', 2), ('m', 2)]


In [35]:
rdd = sc.parallelize([("a", ["apple", "banana", "lemon"]), ("b", ["grapes"])])
print(rdd.mapValues(len).collect())
print(rdd.mapValues(lambda a:a).collect())

[('a', 3), ('b', 1)]
[('a', ['apple', 'banana', 'lemon']), ('b', ['grapes'])]


In [32]:
print(rdd.flatMapValues(lambda a:a).collect())

[('a', 'apple'), ('a', 'banana'), ('a', 'lemon'), ('b', 'grapes')]


In [48]:
def f(a):
    r = [j for i in a for j in i]
    return r
print(rdd.flatMapValues(lambda a:f(a)).collect())

[('a', 'a'), ('a', 'p'), ('a', 'p'), ('a', 'l'), ('a', 'e'), ('a', 'b'), ('a', 'a'), ('a', 'n'), ('a', 'a'), ('a', 'n'), ('a', 'a'), ('a', 'l'), ('a', 'e'), ('a', 'm'), ('a', 'o'), ('a', 'n'), ('b', 'g'), ('b', 'r'), ('b', 'a'), ('b', 'p'), ('b', 'e'), ('b', 's')]


In [49]:
def f(a):
    r = [len(i) for i in a]
    return r
print(rdd.flatMapValues(lambda a:f(a)).collect())

[('a', 5), ('a', 6), ('a', 5), ('b', 6)]


In [52]:
letters = words.flatMap(lambda a: list(a)).map(lambda a:(a,1))
print(letters.aggregateByKey(0,lambda a,b:a+b,lambda a,b:a*b).collect())

# we have to pass initial value, sequence function ie within partition, combiner function ie across or between partitions
# reduce by key is a spl case of aggregate function where both seq and comb funtion are same

[('p', 4), ('l', 2), ('b', 1), ('c', 1), ('r', 8), ('g', 3), ('s', 1), ('a', 24), ('e', 6), ('n', 6), ('o', 4), ('t', 1), ('m', 2)]


In [54]:
letters = words.flatMap(lambda a: list(a)).map(lambda a:(a,1))
print(letters.reduceByKey(lambda a,b:a+b).sortByKey().collect())

[('a', 10), ('b', 1), ('c', 2), ('e', 5), ('g', 3), ('l', 3), ('m', 2), ('n', 5), ('o', 5), ('p', 5), ('r', 6), ('s', 1), ('t', 2)]


In [60]:
a = sc.parallelize([('mango',1),('mango',3)])
b = sc.parallelize([('mango',2)])
c = a.join(b)
print(c.collect())
print(a.groupByKey().mapValues(tuple).collect())

[('mango', (1, 2)), ('mango', (3, 2))]
[('mango', (1, 3))]


In [63]:
print(a.cartesian(b).collect())

[(('mango', 1), ('mango', 2)), (('mango', 3), ('mango', 2))]


In [70]:
a = sc.parallelize([('mango',1),('mango',3)],6)
print(a.getNumPartitions())
b = a.coalesce(3)
print(b.getNumPartitions())
c = b.repartition(8)
print(c.getNumPartitions())

6
3
8


In [71]:
c.glom().collect() #shows partitions

[[], [], [('mango', 1)], [], [], [], [], [('mango', 3)]]

In [ ]:
# Actions

In [77]:
a = sc.parallelize([1,2,3])
a.reduce(lambda x,y:x*y)

6

In [78]:
a.collect()

[1, 2, 3]

In [79]:
a.count()

3

In [80]:
a.take(2)

[1, 2]

In [83]:
a.first()

1

In [85]:
a.saveAsTextFile('/output')

In [ ]:
file = sc.textFile('C:/Users/varsh/OneDrive/Documents/employees.csv')

In [90]:
b = sc.parallelize([(1,2),(1,3),(2,1),(2,2),(2,3)])
b.countByKey()

defaultdict(int, {1: 2, 2: 3})

In [103]:
def f(x):
    print(x)
a.foreach(f) # for sideeffects